In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import os

import dask
#from dask_jobqueue import SLURMCluster
from dask.distributed import Client

import papermill as pm

import oae_smyle
import validate_case

In [21]:
cases = [
    "smyle.control.perlmutter.004",
]
df_caseinfo = oae_smyle.get_caseinfo()
cases += df_caseinfo.loc[(df_caseinfo.run) & (~df_caseinfo.error)].index.to_list()
cases

['smyle.control.perlmutter.004',
 'smyle.oae-map.North_Atlantic_basin.000.1999-04-01.001',
 'smyle.oae-map.North_Atlantic_basin.000.1999-10-01.001',
 'smyle.oae-map.North_Atlantic_basin.001.1999-04-01.001',
 'smyle.oae-map.North_Atlantic_basin.001.1999-07-01.001',
 'smyle.oae-map.North_Atlantic_basin.001.1999-10-01.001',
 'smyle.oae-map.North_Atlantic_basin.002.1999-04-01.001',
 'smyle.oae-map.North_Atlantic_basin.002.1999-07-01.001',
 'smyle.oae-map.North_Atlantic_basin.002.1999-10-01.001',
 'smyle.oae-map.North_Atlantic_basin.003.1999-04-01.001',
 'smyle.oae-map.North_Atlantic_basin.003.1999-07-01.001',
 'smyle.oae-map.North_Atlantic_basin.003.1999-10-01.001',
 'smyle.oae-map.North_Atlantic_basin.004.1999-04-01.001',
 'smyle.oae-map.North_Atlantic_basin.004.1999-07-01.001',
 'smyle.oae-map.North_Atlantic_basin.004.1999-10-01.001',
 'smyle.oae-map.North_Atlantic_basin.005.1999-04-01.001',
 'smyle.oae-map.North_Atlantic_basin.005.1999-10-01.001',
 'smyle.oae-map.North_Atlantic_basin.00

In [22]:
if False:
    USER = os.environ["USER"]
    SCRATCH = f"/pscratch/sd/{USER[0]}/{USER}"
    memory = "128GB"
    walltime = "04:00:00"
    account = "m4746"

    # SLURMCluster doesn't seem to generate at usable jobscript
    # hack it
    job_directives_skip = ["-J", "-e", "-o", "-p","-A", "-n", "--cpus-per-task", "--mem", "-t"]   
    job_extra_directives = [
        "-J dask-worker",
        f"-A {account}",
        f"-e {SCRATCH}/dask-workers/dask-worker-%J.err",
        f"-o {SCRATCH}/dask-workers/dask-worker-%J.out",
        f"--time={walltime}",
        f"--mem={memory}",    
        "--qos=shared",    
        "--nodes=1",
        "--ntasks=16",
        "--ntasks-per-node=64", # shared queue only allows half node — don't know if this is really needed
        "--cpus-per-task=1",
        "--constraint=cpu",
    ]

    cluster = SLURMCluster(
        cores=1,
        processes=1,    
        memory=memory,
        queue="debug",
        account=account,
        walltime=walltime,
        job_directives_skip=job_directives_skip,
        job_extra_directives=job_extra_directives,
    )

    cluster.scale(2)
    client = Client(cluster)

    client

In [23]:
scheduler_file = os.path.join(os.environ["SCRATCH"], "scheduler_file.json")

dask.config.config["distributed"]["dashboard"]["link"] = "{JUPYTERHUB_SERVICE_PREFIX}proxy/{host}:{port}/status" 

client = Client(scheduler_file=scheduler_file)
client

<Client: 'tcp://10.249.10.69:8786' processes=64 threads=16384, memory=29.79 TiB>

In [26]:
clobber = False
for case in cases:
    is_oae_run = ".oae-map." in case
    zarr_store = validate_case.zarr_validation_data(case)
    if not os.path.exists(zarr_store) or clobber:
        validate_case.validator(case, is_oae_run)        

smyle.control.perlmutter.004
smyle.oae-map.North_Atlantic_basin.000.1999-04-01.001
smyle.oae-map.North_Atlantic_basin.000.1999-10-01.001
smyle.oae-map.North_Atlantic_basin.001.1999-04-01.001
smyle.oae-map.North_Atlantic_basin.001.1999-07-01.001
smyle.oae-map.North_Atlantic_basin.001.1999-10-01.001
smyle.oae-map.North_Atlantic_basin.002.1999-04-01.001
smyle.oae-map.North_Atlantic_basin.002.1999-07-01.001
smyle.oae-map.North_Atlantic_basin.002.1999-10-01.001
smyle.oae-map.North_Atlantic_basin.003.1999-04-01.001
smyle.oae-map.North_Atlantic_basin.003.1999-07-01.001
smyle.oae-map.North_Atlantic_basin.003.1999-10-01.001
smyle.oae-map.North_Atlantic_basin.004.1999-04-01.001
smyle.oae-map.North_Atlantic_basin.004.1999-07-01.001
smyle.oae-map.North_Atlantic_basin.004.1999-10-01.001
smyle.oae-map.North_Atlantic_basin.005.1999-04-01.001
smyle.oae-map.North_Atlantic_basin.005.1999-10-01.001
smyle.oae-map.North_Atlantic_basin.006.1999-04-01.001
smyle.oae-map.North_Atlantic_basin.006.1999-10-01.001

In [27]:
clobber = True

dir_out = "case-info/validation"
os.makedirs(dir_out, exist_ok=True)

for case in cases:   
    nb_out = f"{dir_out}/validation_{case}.ipynb"
    if os.path.exists(nb_out) and not clobber:
        continue
    
    zarr_store = validate_case.zarr_validation_data(case)    
    if os.path.exists(zarr_store):
        pm.execute_notebook(
           "plot_case_validation.ipynb",
           nb_out,
           parameters=dict(case=case)
        )

Executing:   0%|          | 0/6 [00:00<?, ?cell/s]

Executing:   0%|          | 0/6 [00:00<?, ?cell/s]

Executing:   0%|          | 0/6 [00:00<?, ?cell/s]

Executing:   0%|          | 0/6 [00:00<?, ?cell/s]

Executing:   0%|          | 0/6 [00:00<?, ?cell/s]

...

Executing:   0%|          | 0/6 [00:00<?, ?cell/s]

Executing:   0%|          | 0/6 [00:00<?, ?cell/s]

Executing:   0%|          | 0/6 [00:00<?, ?cell/s]

Executing:   0%|          | 0/6 [00:00<?, ?cell/s]

Executing:   0%|          | 0/6 [00:00<?, ?cell/s]

In [28]:
print("done")

done
